In [5]:
import tweepy

consumer_key = '3FJT3EvtOssiHYfw2xpH4Q9dR'
consumer_secret = 'OzV47fYGizND0NJlaVYyg0ieoiwmCTHeCi9eMMAKVqCxC9QatT'

access_token = '4879319992-CcCKl9DKwrsZIUItxOeUAA7GgfH5Md8XkyaKYx9'
access_token_secret = 'Rva5W0Ttx1FDRBZW6rtUJT1GRQOjqehOWnFFBKElWrdD5'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

# public_tweets = api.home_timeline()
public_tweets = tweepy.Cursor(api.search, q="#WomensMarch").items(10)
for tweet in public_tweets:
    print (tweet.text)
    print (tweet.created_at)

# # Get the User object for twitter...
# user = api.get_user('twitter')

# # Models contain the data and some helper methods
# print user.screen_name
# print user.followers_count
# for friend in user.friends():
#    print friend.screen_name
    

RT @zlando: When #WomensMarch co-chair (Palestinian Linda Sarsour) tried to bury a movie fighting for #womensrights. WATCH! https://t.co/Fv…
2017-11-14 20:08:54
RT @zlando: When #WomensMarch co-chair (Palestinian Linda Sarsour) tried to bury a movie fighting for #womensrights. WATCH! https://t.co/Fv…
2017-11-14 20:05:57
#ROSEARMY #MeToo #metoomen #metoomarch #WomensMarch Come on feminists, time to actually take a stand on this and do… https://t.co/RwhyHqwlSt
2017-11-14 20:05:36
RT @zlando: When #WomensMarch co-chair (Palestinian Linda Sarsour) tried to bury a movie fighting for #womensrights. WATCH! https://t.co/Fv…
2017-11-14 20:02:33
When #WomensMarch co-chair (Palestinian Linda Sarsour) tried to bury a movie fighting for #womensrights. WATCH!… https://t.co/lzQdjXkSGA
2017-11-14 20:01:02
Want to understand what women, immigrants, POC, LGBQT, and others are so mad about? Listen to these women sing. Rea… https://t.co/KbA0aATkPq
2017-11-14 19:51:38
RT @JDIYFR: Don't Mess with Us --&gt; 